In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import roc_curve, auc, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# -----------------------------
# CONFIGURATION
# -----------------------------
INTEGRATED_FILE = "./tcga_drug_sensitivity_v2_all_drugs/INTEGRATED_all_strategies.tsv"
OUTPUT_DIR = "./validation_results"

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

RESPONSE_MAPPING = {
    "complete response": "Responder",
    "partial response": "Responder",
    "stable disease": "Responder",
    "cr": "Responder",
    "pr": "Responder",
    "sd": "Responder",
    "progressive disease": "Non-responder",
    "clinical progressive disease": "Non-responder",
    "pd": "Non-responder",
    "not evaluable": "Unknown",
    "unknown": "Unknown",
}

SENSITIVITY_THRESHOLD = 0.0

# -----------------------------
# HELPER FUNCTIONS
# -----------------------------
def normalize_response(response):
    if pd.isna(response):
        return "Unknown"
    response_clean = str(response).lower().strip()
    if response_clean in RESPONSE_MAPPING:
        return RESPONSE_MAPPING[response_clean]
    for key, value in RESPONSE_MAPPING.items():
        if key in response_clean:
            return value
    return "Unknown"

def calculate_metrics(y_true, y_pred, y_score=None):
    metrics = {}
    cm = confusion_matrix(y_true, y_pred)
    if cm.size == 4:
        tn, fp, fn, tp = cm.ravel()
    else:
        tn = fp = fn = tp = 0

    denom = tp + tn + fp + fn
    metrics["accuracy"] = (tp + tn) / denom if denom > 0 else 0
    metrics["sensitivity"] = tp / (tp + fn) if (tp + fn) > 0 else 0
    metrics["specificity"] = tn / (tn + fp) if (tn + fp) > 0 else 0
    metrics["ppv"] = tp / (tp + fp) if (tp + fp) > 0 else 0
    metrics["npv"] = tn / (tn + fn) if (tn + fn) > 0 else 0

    if y_score is not None and len(np.unique(y_true)) == 2:
        fpr, tpr, _ = roc_curve(y_true, y_score)
        metrics["auc"] = auc(fpr, tpr)
        metrics["fpr"] = fpr
        metrics["tpr"] = tpr

    return metrics

def plot_roc_curves(results_dict, title):
    fig, ax = plt.subplots(figsize=(10, 8))
    colors = plt.cm.tab20(np.linspace(0, 1, len(results_dict)))

    for (drug, metrics), color in zip(results_dict.items(), colors):
        ax.plot(
            metrics["fpr"],
            metrics["tpr"],
            label=f"{drug} (AUC={metrics['auc']:.3f})",
            linewidth=2,
            color=color,
        )

    ax.plot([0, 1], [0, 1], "k--", linewidth=1)
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.set_title(title)
    ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=8)
    ax.grid(alpha=0.3)
    plt.tight_layout()
    return fig

def plot_score_distribution(df):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    sns.violinplot(
        data=df,
        x="binary_response",
        y="predicted_sensitivity_score_integrated",
        ax=axes[0],
    )
    axes[0].set_title("Score Distribution by Response")
    axes[0].grid(axis="y", alpha=0.3)

    sns.boxplot(
        data=df,
        x="binary_response",
        y="predicted_sensitivity_score_integrated",
        ax=axes[1],
    )
    axes[1].set_title("Score Statistics by Response")
    axes[1].grid(axis="y", alpha=0.3)

    plt.tight_layout()
    return fig

# -----------------------------
# LOAD DATA
# -----------------------------
print("STEP 1: Loading integrated prediction results")

df = pd.read_csv(INTEGRATED_FILE, sep="\t")
print(f"Total samples: {len(df)}")
print(f"Columns: {list(df.columns)}")

# -----------------------------
# CLEAN CLINICAL RESPONSE
# -----------------------------
print("STEP 2: Cleaning clinical response labels")

df["response_cleaned"] = df["clinical_response"].apply(normalize_response)
df_valid = df[df["response_cleaned"] != "Unknown"].copy()

if len(df_valid) == 0:
    raise ValueError("No valid clinical response labels found")

df_valid["binary_response"] = (df_valid["response_cleaned"] == "Responder").astype(int)

# -----------------------------
# OVERALL T-TEST
# -----------------------------
print("STEP 3: Overall t-test analysis")

resp = df_valid[df_valid["binary_response"] == 1]["predicted_sensitivity_score_integrated"]
non_resp = df_valid[df_valid["binary_response"] == 0]["predicted_sensitivity_score_integrated"]

t_stat, p_value = stats.ttest_ind(resp, non_resp)

pooled_std = np.sqrt(
    ((len(resp) - 1) * resp.std() ** 2 + (len(non_resp) - 1) * non_resp.std() ** 2)
    / (len(resp) + len(non_resp) - 2)
)
cohens_d = (resp.mean() - non_resp.mean()) / pooled_std

print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.6f}")
print(f"Cohen_d: {cohens_d:.4f}")

# -----------------------------
# CLASSIFICATION METRICS
# -----------------------------
print("STEP 4: Classification performance")

df_valid["predicted_class"] = (
    df_valid["predicted_sensitivity_score_integrated"] > SENSITIVITY_THRESHOLD
).astype(int)

overall_metrics = calculate_metrics(
    df_valid["binary_response"],
    df_valid["predicted_class"],
    df_valid["predicted_sensitivity_score_integrated"],
)

for k, v in overall_metrics.items():
    if k not in ["fpr", "tpr"]:
        print(f"{k}: {v}")

# -----------------------------
# PER-DRUG ANALYSIS
# -----------------------------
print("STEP 5: Per-drug analysis")

drug_results = []
drug_metrics = {}

for drug in df_valid["drug_name"].unique():
    sub = df_valid[df_valid["drug_name"] == drug]
    if len(sub) < 10 or sub["binary_response"].nunique() < 2:
        continue

    r = sub[sub["binary_response"] == 1]["predicted_sensitivity_score_integrated"]
    nr = sub[sub["binary_response"] == 0]["predicted_sensitivity_score_integrated"]

    t_stat, p_val = stats.ttest_ind(r, nr)
    fpr, tpr, _ = roc_curve(sub["binary_response"], sub["predicted_sensitivity_score_integrated"])
    roc_auc = auc(fpr, tpr)

    drug_results.append({
        "drug": drug,
        "n_total": len(sub),
        "mean_diff": r.mean() - nr.mean(),
        "p_value": p_val,
        "auc": roc_auc,
    })

    drug_metrics[drug] = {"fpr": fpr, "tpr": tpr, "auc": roc_auc}

drug_df = pd.DataFrame(drug_results).sort_values("p_value")
drug_df.to_csv(f"{OUTPUT_DIR}/per_drug_analysis.csv", index=False)

# -----------------------------
# PER-CANCER ANALYSIS
# -----------------------------
print("STEP 6: Per-cancer analysis")

cancer_results = []

for cancer in df_valid["cancer"].dropna().unique():
    sub = df_valid[df_valid["cancer"] == cancer]
    if len(sub) < 10 or sub["binary_response"].nunique() < 2:
        continue

    r = sub[sub["binary_response"] == 1]["predicted_sensitivity_score_integrated"]
    nr = sub[sub["binary_response"] == 0]["predicted_sensitivity_score_integrated"]

    t_stat, p_val = stats.ttest_ind(r, nr)
    fpr, tpr, _ = roc_curve(sub["binary_response"], sub["predicted_sensitivity_score_integrated"])
    roc_auc = auc(fpr, tpr)

    cancer_results.append({
        "cancer": cancer,
        "n_total": len(sub),
        "mean_diff": r.mean() - nr.mean(),
        "p_value": p_val,
        "auc": roc_auc,
    })

cancer_df = pd.DataFrame(cancer_results).sort_values("p_value")
cancer_df.to_csv(f"{OUTPUT_DIR}/per_cancer_analysis.csv", index=False)

# -----------------------------
# VISUALIZATION
# -----------------------------
print("STEP 7: Generating plots")

fig1 = plot_score_distribution(df_valid)
fig1.savefig(f"{OUTPUT_DIR}/score_distribution.png", dpi=300, bbox_inches="tight")

top_drugs = drug_df.head(5)["drug"].tolist()
top_metrics = {k: v for k, v in drug_metrics.items() if k in top_drugs}

fig2 = plot_roc_curves(top_metrics, "ROC Curves - Top Drugs")
fig2.savefig(f"{OUTPUT_DIR}/roc_curves_top_drugs.png", dpi=300, bbox_inches="tight")

plt.close("all")

print("Validation pipeline completed")

STEP 1: 載入集成預測結果
✓ 總樣本數: 1232
✓ 欄位: ['patient_id', 'drug_name', 'cancer', 'clinical_response', 'predicted_sensitivity_score_integrated', 'n_strategies', 'avg_n_genes_up', 'avg_n_genes_down']

STEP 2: 清理臨床反應標籤

原始標籤分布:
clinical_response
Complete Response               720
Clinical Progressive Disease    318
Stable Disease                  106
Partial Response                 88
Name: count, dtype: int64

清理後標籤分布:
response_cleaned
Responder        914
Non-responder    318
Name: count, dtype: int64

✓ 有效樣本數: 1232 (100.0%)

STEP 3: 整體 T-test 分析

Responders (n=914):
  Mean: 0.0024
  Std:  0.1371

Non-responders (n=318):
  Mean: -0.0198
  Std:  0.1550

T-statistic: 2.4041
P-value:     0.016361
Significant: YES ✓
Effect Size (Cohen's d): 0.1565

STEP 4: 分類性能評估

整體分類指標 (threshold=0.0):
  Accuracy:    0.537
  Sensitivity: 0.533
  Specificity: 0.550
  PPV:         0.773
  NPV:         0.291
  AUC:         0.543

Confusion Matrix:
[[175 143]
 [427 487]]
[[TN=175  FP=143]
 [FN=427  TP=487]]

STEP 